In [2]:
import pandas as pd

In [34]:
#Cargamos las bases de datos

spam = pd.read_csv("http://turing.iimas.unam.mx/~gibranfp/cursos/aprendizaje_automatizado/data/spam.csv", header=None)
tumores= pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data", header=None)

In [ ]:
# Separamos las columnas de spam
df_split = spam[0].str.split(expand=True)

# Renombramos las columnas
df_split.columns = [f'col_{i}' for i in range(df_split.shape[1]-1)] + ['target']

# Convertimos las columnas a numéricas
df_split = df_split.apply(pd.to_numeric)

In [5]:
y = df_split['target']           # Variable dependiente
X = df_split.drop(columns=['target'])  # Variables independientes

In [ ]:
#Cargamos la función de regresión logística
from scipy.optimize import minimize

class LogisticRegressionML:
    def fit(self, X, y):
        """
        Estima los parámetros (pesos) por máxima verosimilitud
        """
        self.clases = np.unique(y)
        if len(self.clases) != 2:
            raise ValueError("Este modelo solo soporta clasificación binaria.")
        
        n = X.shape[0]
        self.n_atr = X.shape[1]

        # Insertamos el bias (columna de unos)
        X = np.insert(X, 0, 1, axis=1)
        self.X_train = X
        self.y_train = y
        w_init = np.zeros(X.shape[1])

        def sigmoid(z):
            return 1 / (1 + np.exp(-z))

        def loss(w):
            z = X @ w
            h = sigmoid(z)
            epsilon = 1e-5
            return -np.mean(y * np.log(h + epsilon) + (1 - y) * np.log(1 - h + epsilon))

        def gradient(w):
            z = X @ w
            h = sigmoid(z)
            return X.T @ (h - y) / n

        res = minimize(fun=loss, x0=w_init, jac=gradient, method='BFGS')
        self.w = res.x

    def predict_proba(self, X):
        """
        Calcula la probabilidad de la clase positiva (label 1)
        """
        X = np.insert(X, 0, 1, axis=1)
        z = X @ self.w
        probs = 1 / (1 + np.exp(-z))
        return np.column_stack([1 - probs, probs])  # clase 0 y clase 1

    def predict(self, X):
        """
        Predice la clase (0 o 1) según el umbral 0.5
        """
        probs = self.predict_proba(X)
        return (probs[:, 1] >= 0.5).astype(int)


In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import numpy as np

# Definimos el clasificador
pipeline = make_pipeline(StandardScaler(), LogisticRegressionML())

#Creamos el objeto de validación cruzada
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

#Calculamos los scores
scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')

# Mostramos los resultados
print(f"Puntaje promedio: {scores.mean():.4f}")
print(f"Desviación estándar: {scores.std():.4f}")


Puntaje promedio: 0.9703
Desviación estándar: 0.0046


In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline

# Definimos el clasificador 
pipeline = make_pipeline(GaussianNB())

# Creamos el objeto de validación cruzada
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

# Calculamos los scores
scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')

# Mostramos los resultados
print(f"Puntaje promedio: {scores.mean():.4f}")
print(f"Desviación estándar: {scores.std():.4f}")


Puntaje promedio: 0.9204
Desviación estándar: 0.0081


In [36]:
# Definimos los nombres de las columnas
column_names = [
    "ID_Muestra", "Grosor_Tumor", "Uniformidad_Tamaño_Celular", 
    "Uniformidad_Forma_Celular", "Adhesión_Marginal", "Tamaño_Célula_Epitelial", 
    "Núcleos_Desnudos", "Cromatina_Blanda", "Nucléolos_Normales", 
    "Mitosis_Celular", "Clase"
]

# Asignamos los nombres de las columnas
tumores.columns = column_names

In [37]:
#quitaremosw el ID de la muestra ya que no aporta información relevante
tumores.drop(columns="ID_Muestra", inplace=True)


In [38]:
#1 eliminar las filas con valores nulos 
tumores['Núcleos_Desnudos'] = pd.to_numeric(tumores['Núcleos_Desnudos'], errors='coerce')  # Convierte a numérico y mete NaN donde hay '?'
tumores.dropna(inplace=True)  # Elimina filas con NaN


In [ ]:
#Mapeamos la clase 2 a 0 y la clase 4 a 1
tumores['Clase'] = tumores['Clase'].map({2: 0, 4: 1})


In [40]:
y = tumores['Clase']           # Variable dependiente
X = tumores.drop(columns=['Clase'])  # Variables independientes

In [ ]:

# Definimos el clasificador
pipeline = make_pipeline(StandardScaler(), LogisticRegressionML())

#Creamos el objeto de validación cruzada
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

#Calculamos los scores
scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')

# Mostramos los resultados
print(f"Puntaje promedio: {scores.mean():.4f}")
print(f"Desviación estándar: {scores.std():.4f}")

Puntaje promedio: 0.9679
Desviación estándar: 0.0124


In [42]:
# Definimos el clasificador (sin escalado)
pipeline = make_pipeline(GaussianNB())

# Creamos el objeto de validación cruzada
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

# Calculamos los scores
scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')

# Mostramos los resultados
print(f"Puntaje promedio: {scores.mean():.4f}")
print(f"Desviación estándar: {scores.std():.4f}")

Puntaje promedio: 0.9621
Desviación estándar: 0.0142


En primera instancia uno hubiera pensado que el clasificador bayesiano ingenuo era una buena opción para clasificar los datos de spam, sin embargo, los resultados no fueron favorables para el modelo, el clasificador de regresión logística fue mejor tanto en la clasificación binaria de electos, con los tumores, como en el clasificador de lo que pudiera parecer la matriz de apariciones de textos.

Esto se explica, ya que en ambos modelos el supuesto de independencia no se sostiene, ya que en el lenguaje la aparición de ciertas palabras cambia la probabilidad de ocurrencia de otras.

Por lo tanto, vemos que la regresión logística funciona mejor como un elemento de segmentación binaria, dados los resultados, por ende sería el modelo que seleccionaríamos.